<a id='top'></a>

<a href='#import'>Import &amp; Inspect</a> | <a href='#gda'>Graphical Data Analysis</a> | <a href='#nav2'>Nav Two</a> | <a href='#nav3'>Nav Three</a> | <a href='#nav4'>Nav Four</a> | <a href='#nav5'>Nav Five</a> | <a href='#nav6'>Nav Six</a>
    
<br>

---


# Academic Mastery Study - Exploratory Data Analysis

**Improve academic evaluations by predicting whether a student will answer the next test question correctly.** Students taking advantage of practice exercises through various software applications seek an evaluation of their preparedness for formal academic assessments. They would like an an accurate evaluation of any gaps in knowledge and skills without sitting through exercises that repeatedly test established proficiencies and difficulties. The goal is to help test developers build an application that precisely evaluates competencies so that fewer demonstrations of proficiency can accurately determine mastery.

## DATA WRANGLING
---
The [`initial_data_exploration`](initial_data_exploration.ipynb) notebook contains the following work:

Download the `grockit_all_data.zip` data file from the [Grockit competition on Kaggle](https://www.kaggle.com/c/WhatDoYouKnow/data). Import and inspect the raw training.csv data, isolate relevant variables, calculate new variables, organize the dataframe, and resolve missing, invalid, corrupted or duplicate values. Begin creating data visualizations to inspect variables and analyze outliers.

From that notebook, the clean dataframe was saved in the file `clean_training.csv`, which is used for work in this notebook.

## EXPLORATORY DATA ANALYSIS
---
Continue with data visualization, using plots to gain insights, look for correlations, consider a hypothesis to explore, then leverage statistical inference to test the hypothesis.

<a href='#top' id='import'>back to menu</a>

### Import Packages & Clean Training Dataset

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from datetime import datetime
from scipy import stats

# Show all plots:
%matplotlib inline

In [2]:
# Read and save CSV file a as dataframe:
df = pd.read_csv('../data/interim/clean_training.csv')

### Inspect Clean Data

In [3]:
# Inspect the current state of the dataset:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4679166 entries, 0 to 4679165
Data columns (total 16 columns):
Unnamed: 0          int64
user_id             int64
round_started_at    object
answered_at         object
deactivated_at      object
round_ended_at      object
question_id         int64
correct             int64
group_name          int64
track_name          int64
subtrack_name       int64
tag_ids             object
question_type       int64
game_type           int64
num_players         int64
round_duration      object
dtypes: int64(10), object(6)
memory usage: 571.2+ MB


An extra column, `Unnamed: 0`, appears in the new dataframe. This is inspected next. Relevant variables need to be converted to timedate, timedelta and category types.

In [4]:
# Inspect the structure and first 5 observations of the clean dataset:
df.head()

,Unnamed: 0,user_id,round_started_at,answered_at,deactivated_at,round_ended_at,question_id,correct,group_name,track_name,subtrack_name,tag_ids,question_type,game_type,num_players,round_duration
0,0,85818,2010-08-18 20:17:13,2010-08-18 20:18:18,2010-08-18 20:18:18,2010-08-18 20:18:18,5560,0,1,5,14,"['222', '233', '240', '246']",0,7,1,0 days 00:01:05.000000000
1,1,85818,2010-08-18 20:19:12,2010-08-18 20:20:34,2010-08-18 20:20:34,2010-08-18 20:20:34,4681,1,1,5,0,"['24', '49']",0,7,1,0 days 00:01:22.000000000
2,2,85818,2010-08-18 20:20:42,2010-08-18 20:21:56,2010-08-18 20:21:56,2010-08-18 20:21:56,1529,1,1,5,0,"['31', '49']",0,7,1,0 days 00:01:14.000000000
3,3,85818,2010-08-18 20:22:03,2010-08-18 20:23:05,2010-08-18 20:23:05,2010-08-18 20:23:05,2908,1,1,5,0,"['31', '49']",0,7,1,0 days 00:01:02.000000000
4,4,85818,2010-08-18 20:23:11,2010-08-18 20:26:08,2010-08-18 20:26:08,2010-08-18 20:26:08,1773,1,1,5,11,"['183', '194', '207']",0,7,1,0 days 00:02:57.000000000


In [5]:
# Inspect the last 5 observations:
df.tail()

,Unnamed: 0,user_id,round_started_at,answered_at,deactivated_at,round_ended_at,question_id,correct,group_name,track_name,subtrack_name,tag_ids,question_type,game_type,num_players,round_duration
4679161,4851470,58569,2009-10-18 17:08:32,2009-10-18 17:09:06,2009-10-18 17:09:06,2009-10-18 17:09:06,1633,1,1,4,9,"['47', '82', '118', '161']",0,7,1,0 days 00:00:34.000000000
4679162,4851471,58569,2009-10-18 17:09:24,NaN,2009-10-18 17:10:03,2009-10-18 17:10:03,1775,0,1,4,1,"['4', '52', '135', '262']",0,7,1,0 days 00:00:39.000000000
4679163,4851472,131842,2010-04-19 23:13:02,NaN,2010-04-19 23:13:37,2010-04-19 23:13:37,1655,0,2,8,5,"['106', '130', '164']",0,7,5,0 days 00:00:35.000000000
4679164,4851473,131842,2010-04-19 23:13:48,NaN,2010-04-19 23:14:35,2010-04-19 23:14:35,544,0,2,8,3,"['101', '103', '134', '280']",0,7,5,0 days 00:00:47.000000000
4679165,4851474,52513,2010-05-18 14:32:59,NaN,2010-05-18 14:35:15,2010-05-18 14:35:15,1272,0,2,7,13,"['239', '245']",0,7,1,0 days 00:02:16.000000000


The column, `Unnamed: 0`, looks like it was the index of the clean dataframe before it was saved as a CSV file in the previous notebook. The last value corresponds to the last index value before observations were eliminated. This column is dropped.

In [6]:
# Drop Unnamed: 0 columns and save results:
df = df.drop('Unnamed: 0', axis='columns')

In [7]:
# Create and save list of columns to convert to datetime:
time_columns = ['round_started_at', 'answered_at', 'deactivated_at', 'round_ended_at']

# Loop through the time_columns list:
for column in time_columns:
    # Convert date columns to datetime objects; 
    # need to ‘coerce’ errors, since there are known NULL values:
    df[column] = pd.to_datetime(df[column], errors='coerce')

In [8]:
# Convert round_duration to timedelta:
df['round_duration'] = pd.to_timedelta(df.round_duration)

In [9]:
# Define a generic replace function -

def code_values(column, code_dictionary):
    """Return a column of codes defined in a dictionary."""
    
    # Initialize the replacement column:
    coded_column = pd.Series(column, copy=True)
    
    # Loop throught the provided dictionary:
    for key, value in code_dictionary.items():
        
        # Replace key with the value within the original column:
        coded_column.replace(key, value, inplace=True)
        
    return coded_column

In [10]:
# Add a more semantic column related to the 'correct' column -

# Save dictionary with codes and semantic values:
outcome = {0: 'incorrect', 1: 'correct'}

# Copy the 'correct' column and save, then convert:
df['outcome'] = df.correct
df.outcome = code_values(df.outcome, outcome)

# Confirm the changes:
df.head()

,user_id,round_started_at,answered_at,deactivated_at,round_ended_at,question_id,correct,group_name,track_name,subtrack_name,tag_ids,question_type,game_type,num_players,round_duration,outcome
0,85818,2010-08-18 20:17:13,2010-08-18 20:18:18,2010-08-18 20:18:18,2010-08-18 20:18:18,5560,0,1,5,14,"['222', '233', '240', '246']",0,7,1,00:01:05,incorrect
1,85818,2010-08-18 20:19:12,2010-08-18 20:20:34,2010-08-18 20:20:34,2010-08-18 20:20:34,4681,1,1,5,0,"['24', '49']",0,7,1,00:01:22,correct
2,85818,2010-08-18 20:20:42,2010-08-18 20:21:56,2010-08-18 20:21:56,2010-08-18 20:21:56,1529,1,1,5,0,"['31', '49']",0,7,1,00:01:14,correct
3,85818,2010-08-18 20:22:03,2010-08-18 20:23:05,2010-08-18 20:23:05,2010-08-18 20:23:05,2908,1,1,5,0,"['31', '49']",0,7,1,00:01:02,correct
4,85818,2010-08-18 20:23:11,2010-08-18 20:26:08,2010-08-18 20:26:08,2010-08-18 20:26:08,1773,1,1,5,11,"['183', '194', '207']",0,7,1,00:02:57,correct


In [11]:
# Create and save list of columns to convert to categorical data type:
category_columns = [
    'user_id',
    'question_id', 
    'group_name', 
    'track_name', 
    'subtrack_name', 
    'question_type', 
    'game_type', 
    'num_players',
    'outcome'
]

# Loop through the category_columns list:
for column in category_columns:
    # Convert column to category type:
    df[column] = df[column].astype('category')

In [12]:
# Confirm the count of users:
print( ("Number of Users: "), len(df.user_id.value_counts()) )

Number of Users:  178342


In [13]:
# Confirm the count of unique questions:
print(("Number of Unique Questions: "), (len( df.question_id.unique() )))

Number of Unique Questions:  6045


In [14]:
# Confirm the null count:
df.isnull().sum() 

user_id                  0
round_started_at         0
answered_at         511154
deactivated_at          10
round_ended_at           0
question_id              0
correct                  0
group_name               0
track_name               0
subtrack_name            0
tag_ids                  0
question_type            0
game_type                0
num_players              0
round_duration           0
outcome                  0
dtype: int64

There are 44,844 fewer NULL values in this dataset than indicated in the last inspection of such in the previous notebook, but that count was obtained before more records were dropped for other reasons related to time. 

Again, observations where both `answered_at` and `deactivated_at` values are NULL were eliminated. Either variable provides the necessary information for `round_ended_at`, so ignoring one of the two is not a problem.

In [15]:
# Re-inspect the state and structure of the dataframe:
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4679166 entries, 0 to 4679165
Data columns (total 16 columns):
user_id             category
round_started_at    datetime64[ns]
answered_at         datetime64[ns]
deactivated_at      datetime64[ns]
round_ended_at      datetime64[ns]
question_id         category
correct             int64
group_name          category
track_name          category
subtrack_name       category
tag_ids             object
question_type       category
game_type           category
num_players         category
round_duration      timedelta64[ns]
outcome             category
dtypes: category(9), datetime64[ns](4), int64(1), object(1), timedelta64[ns](1)
memory usage: 314.5+ MB
None


,user_id,round_started_at,answered_at,deactivated_at,round_ended_at,question_id,correct,group_name,track_name,subtrack_name,tag_ids,question_type,game_type,num_players,round_duration,outcome
0,85818,2010-08-18 20:17:13,2010-08-18 20:18:18,2010-08-18 20:18:18,2010-08-18 20:18:18,5560,0,1,5,14,"['222', '233', '240', '246']",0,7,1,00:01:05,incorrect
1,85818,2010-08-18 20:19:12,2010-08-18 20:20:34,2010-08-18 20:20:34,2010-08-18 20:20:34,4681,1,1,5,0,"['24', '49']",0,7,1,00:01:22,correct
2,85818,2010-08-18 20:20:42,2010-08-18 20:21:56,2010-08-18 20:21:56,2010-08-18 20:21:56,1529,1,1,5,0,"['31', '49']",0,7,1,00:01:14,correct
3,85818,2010-08-18 20:22:03,2010-08-18 20:23:05,2010-08-18 20:23:05,2010-08-18 20:23:05,2908,1,1,5,0,"['31', '49']",0,7,1,00:01:02,correct
4,85818,2010-08-18 20:23:11,2010-08-18 20:26:08,2010-08-18 20:26:08,2010-08-18 20:26:08,1773,1,1,5,11,"['183', '194', '207']",0,7,1,00:02:57,correct


<a href='#top' id='gda'>back to menu</a>

### Graphical Data Analysis



In [16]:
# Filter columns for category datatype and display the statistics:
categorical = df.dtypes[df.dtypes == 'category'].index
df[categorical].describe()

,user_id,question_id,group_name,track_name,subtrack_name,question_type,game_type,num_players,outcome
count,4679166,4679166,4679166,4679166,4679166,4679166,4679166,4679166,4679166
unique,178342,6045,3,9,16,2,11,33,2
top,133472,4059,1,5,9,0,7,1,correct
freq,8465,14281,2319061,1204729,700932,4498607,2993960,3095229,2593402


In [17]:
# Replace group codes with semantic values -

# Save dictionary with codes and semantic values:
groups = {0:'act', 1:'gmat', 2:'sat'}

# Convert and save:
df.group_name = code_values(df.group_name, groups)

In [18]:
# Get a view of tiered variables related to question groups:
group_tracks = pd.crosstab(index=df.outcome,columns=[df.group_name, df.track_name, df.subtrack_name])
group_tracks

group_name        act                          gmat                          \
track_name          0       1      2      3       4               5           
subtrack_name      2       6      10     12      1       9       0       11   
outcome                                                                       
correct        169228  110165  35272  43018  226528  381546  209012  156588   
incorrect      139286  119679  36223  43598  186872  319386  152612  129556   

group_name                sat                                                
track_name                  6              7               8                 
subtrack_name      14      7       15     8       13      3      4       5   
outcome                                                                      
correct        340268  251627   76802  86571  172787  131840  41576  160574  
incorrect      216693  178842  103757  81790  119504  106225  38690  113051

The most helpful information in the chart above is confirmation that there is no overlap in the tracks or subtracks per group. It may be useful to split the dataframe (for now) by group name.